# Collision Avoidance - Train Model

In this notebook we'll train our image classifier to detect 3 classes
``free``, ``blocked``, and ``caution``. which we'll use for drive the JetBot around the living room.  For this, we'll use a popular deep learning library *PyTorch*

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

### Dataset

Before you start, you should complete the data collection notebook and have a ``dataset_FBC`` folder in the directory.

### Create dataset instance

Now we use the ``ImageFolder`` dataset class available with the ``torchvision.datasets`` package.  We attach transforms from the ``torchvision.transforms`` package to prepare the data for training.  

In [2]:
dataset = datasets.ImageFolder(
    'dataset_FB',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

### Split dataset into train and test sets

Next, we split the dataset into *training* and *test* sets.  The test set will be used to verify the accuracy of the model we train.

In [3]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 50, 50])

### Create data loaders to load data in batches

We'll create two ``DataLoader`` instances, which provide utilities for shuffling data, producing *batches* of images, and loading the samples in parallel with multiple workers.

In [4]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

### Define the neural network

Now, we define the neural network we'll be training.  The *torchvision* package provides a collection of pre-trained models that we can use.

In a process called *transfer learning*, we can repurpose a pre-trained model (trained on millions of images) for a new task that has possibly much less data available.

Important features that were learned in the original training of the pre-trained model are re-usable for the new task.  We'll use the ``alexnet`` model.

In [5]:
model = models.alexnet(pretrained=True)

The ``alexnet`` model was originally trained for a dataset that had 1000 class labels, but our dataset only has only four class labels!  We'll replace
the final layer with a new, untrained layer that has only four outputs (``free``and ``blocked``.)  

In [6]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2) # 2 here refers to the number of outputs (class labels)

Finally, we transfer our model for execution on the GPU

In [7]:
device = torch.device('cuda')
model = model.to(device)

### Train the neural network

Using the code below we will train the neural network for 50 epochs, saving the best performing model after each epoch.

> An epoch is a full run through our data.

In [8]:
NUM_EPOCHS = 50
BEST_MODEL_PATH = 'best_model_FB.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

0: 0.960000
1: 0.960000
2: 0.960000
3: 0.980000
4: 1.000000
5: 0.980000
6: 0.960000
7: 0.980000
8: 0.980000
9: 1.000000
10: 1.000000
11: 0.960000
12: 0.960000
13: 1.000000
14: 0.960000
15: 0.980000
16: 0.980000
17: 0.940000
18: 0.960000
19: 0.960000
20: 0.940000
21: 1.000000
22: 0.980000
23: 0.980000
25: 0.980000
26: 0.980000
27: 0.980000
28: 0.980000
29: 0.980000
30: 1.000000
31: 0.980000
32: 0.980000
33: 1.000000
34: 0.960000
35: 0.980000
36: 0.980000
37: 1.000000
38: 1.000000
39: 0.980000
40: 0.980000
41: 0.980000
42: 1.000000
43: 0.980000
44: 0.980000
45: 0.980000
46: 1.000000
47: 1.000000
48: 0.960000
49: 0.980000


Note: An error will occur if the number of folders within the ``dataset_FBC`` directory doesn't match the number specified by the ``model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 3)`` line of code. The "3" refers to our three datasets, ``free``, ``blocked``, and ``caution``. First, double-check that you've specified the correct number. If so, it is likely that a hidden folder has been inserted into your ``dataset_FBC`` directory by Jupyter Notebooks. Open a terminal window, navigate to the directory, and type ``ls -a`` to list all files and folders. Remove the ``.ipynb_checkpoints`` directory and restart this notebook from the top.

Once that is finished, you should see a file ``best_model_FBC.pth`` in the Jupyter Lab file browser. You will use that model for the ``run_FBC`` notebook.